In [57]:
# %pip install tensorflow[and-cuda]

In [58]:

import numpy as np
import os
import pandas as pd
from tensorflow.keras.utils import image_dataset_from_directory
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow import keras

from tensorflow.keras.layers.experimental import preprocessing
from keras.preprocessing import image
from keras.applications.vgg16 import VGG16, preprocess_input
from keras.models import Model



In [59]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
import matplotlib.pyplot as plt


In [60]:
def load_cnn():
  cnn = VGG16(weights='imagenet', include_top=True)
  return cnn



In [61]:
model = load_cnn()
model.summary()


Model: "vgg16"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_9 (InputLayer)        [(None, 224, 224, 3)]     0         
                                                                 
 block1_conv1 (Conv2D)       (None, 224, 224, 64)      1792      
                                                                 
 block1_conv2 (Conv2D)       (None, 224, 224, 64)      36928     
                                                                 
 block1_pool (MaxPooling2D)  (None, 112, 112, 64)      0         
                                                                 
 block2_conv1 (Conv2D)       (None, 112, 112, 128)     73856     
                                                                 
 block2_conv2 (Conv2D)       (None, 112, 112, 128)     147584    
                                                                 
 block2_pool (MaxPooling2D)  (None, 56, 56, 128)       0     

In [62]:
def get_datasets():
  training_dataset, test_dataset = image_dataset_from_directory(
    '/content/drive/Shareddrives/AADT/images',
     labels="inferred",
    label_mode="int",
    class_names=None,
    color_mode="rgb",
    batch_size=32,
    image_size=(224, 224),
    shuffle=True,
    seed=7676,
    validation_split=0.2,
    subset="both",
    interpolation="bilinear",
    follow_links=False,
    crop_to_aspect_ratio=False,
  )
  return training_dataset, test_dataset





In [63]:
import re


def getId(pt):
  match = re.search(r'i([^_/]+)_([^\.]+)\.png$', pt)
  if match:
      p1 = match.group(1)
      p2 = match.group(2)
      result = f"{p1}_{p2}"
  else:
      result = None
  return result

In [64]:
# import os
# import shutil

# # Define source and destination directories
# source_dir = '/content/drive/Shareddrives/AADT/images/images'
# dest_dir = '/content/drive/Shareddrives/AADT/imgs'

# # Ensure the destination directory exists
# os.makedirs(dest_dir, exist_ok=True)
# print(len(os.listdir(source_dir)))
# # Iterate over all files in the source directory
# for filename in os.listdir(source_dir):
#     print(filename)
#     if filename.endswith('.png'):
#         # Extract the p1 part of the filename
#         p1 = getId(filename)  # Assuming filename is of the form i<p1>.png

#         # Define the new directory path
#         new_dir = os.path.join(dest_dir, p1)

#         # Create the new directory if it doesn't exist
#         os.makedirs(new_dir, exist_ok=True)

#         # Define the full file paths
#         src_file = os.path.join(source_dir, filename)
#         dest_file = os.path.join(new_dir, filename)

#         # Move the file
#         shutil.move(src_file, dest_file)

In [65]:
full_count_df = pd.read_csv('full_count.csv')
classes = sorted(list(os.listdir('image_folders/')))
classes = [c.replace("_", "@") for c in classes]
classes


['000001@01',
 '000001@08',
 '000001@13',
 '000001@16',
 '000001@22',
 '000001@25',
 '000001@31',
 '000001@42',
 '000001@45',
 '000001@54',
 '000002@01',
 '000002@08',
 '000002@13',
 '000002@15',
 '000002@16',
 '000002@22',
 '000002@25',
 '000002@31',
 '000002@42',
 '000002@45',
 '000002@54',
 '000003@08',
 '000003@13',
 '000003@15',
 '000003@16',
 '000003@22',
 '000003@25',
 '000003@31',
 '000003@45',
 '000003@47',
 '000003@54',
 '000004@08',
 '000004@15',
 '000004@16',
 '000004@22',
 '000004@31',
 '000004@42',
 '000004@45',
 '000004@47',
 '000004@48',
 '000004@54',
 '000005@08',
 '000005@16',
 '000005@25',
 '000005@31',
 '000005@42',
 '000005@45',
 '000005@47',
 '000006@08',
 '000006@15',
 '000006@16',
 '000006@25',
 '000006@31',
 '000006@45',
 '000006@46',
 '000006@47',
 '000006@48',
 '000006@54',
 '000007@01',
 '000007@08',
 '000007@16',
 '000007@22',
 '000007@25',
 '000007@31',
 '000007@39',
 '000007@42',
 '000007@45',
 '000007@47',
 '000007@48',
 '000007@54',
 '000008@08',
 '0000

In [66]:
def get_average_by_id( id):
    id = classes[id]
    return df.loc[full_count_df['id'] == id, 'average'].values[0]

In [67]:
def get_complete_dataset():
  dataset = image_dataset_from_directory(
      'image_folders',
      labels="inferred",
      label_mode="int",
      class_names=None,
      color_mode="rgb",
      batch_size=32,
      image_size=(224, 224),
      shuffle=True,
      seed=7676,
      validation_split=0,
      interpolation="bilinear",
      follow_links=False,
      crop_to_aspect_ratio=False,
  )
  gen = ImageDataGenerator(
    horizontal_flip=True,
    vertical_flip=True,

  )
  # Define the augmentation layer
  data_augmentation = tf.keras.Sequential([
      preprocessing.RandomFlip("horizontal_and_vertical"),
      preprocessing.RandomRotation(
        (-1.57079633 , 1.57079633)
    )
  ])

  # Apply the augmentation to the dataset
  def augment(images, labels):
      images = data_augmentation(images)
      # labels = classes[labels]
      return images, labels

  augmented_dataset = dataset.map(augment)

  return augmented_dataset

In [68]:
# os.listdir('/content/drive/Shareddrives/AADT/image_folders/000001_13')

In [69]:
dataset = get_complete_dataset()
cnn= load_cnn()

features=[]
labels= []
for step in range(2):
  print("======", step, "======")
  for img_batch, label_batch in dataset:

      labels.append(label_batch)
      x = preprocess_input(img_batch)
      p = cnn.predict(x)
      # shape = p.shape

      # # Calculate M
      # M = shape[1] * shape[2] * shape[3]
      features.append(p)
      print("step")



labels = tf.concat(labels, axis=0)
features = tf.concat(features, axis=0)
labels
files = labels

Found 4820 files belonging to 4820 classes.
====== 0 ======
1/1 [==============================] - 2s 2s/step
step
1/1 [==============================] - 2s 2s/step
step
1/1 [==============================] - 3s 3s/step
step
1/1 [==============================] - 2s 2s/step
step
1/1 [==============================] - 3s 3s/step
step
1/1 [==============================] - 2s 2s/step
step
1/1 [==============================] - 2s 2s/step
step
1/1 [==============================] - 2s 2s/step
step
1/1 [==============================] - 3s 3s/step
step
1/1 [==============================] - 2s 2s/step
step
1/1 [==============================] - 2s 2s/step
step
1/1 [==============================] - 2s 2s/step
step
1/1 [==============================] - 2s 2s/step
step
1/1 [==============================] - 3s 3s/step
step
1/1 [==============================] - 3s 3s/step
step
1/1 [==============================] - 3s 3s/step
step
1/1 [==============================] - 3s 3s/step
step
1/1 [

KeyboardInterrupt: 

In [ ]:
labels

<tf.Tensor: shape=(24100,), dtype=int32, numpy=array([4593, 2705,  906, ..., 3551, 2983, 1613])>

In [ ]:
ids = [ classes[f] for f in files ]
ids


[<tf.Tensor: shape=(), dtype=string, numpy=b'0R3102@37'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'004203@29'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'000153@45'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'00034B@34'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'0W7501@37'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'001531@36'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'006480@36'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'000415@25'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'000052@31'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'000020@45'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'080499@51'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'0A2501@37'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'005031@29'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'100118@04'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'000280@13'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'000506@42'>,
 <tf.Tensor: shape=(), dtype=string, numpy=b'00Y006@50'>,
 <tf.Tensor: s

In [ ]:
df = pd.DataFrame(data=features)
str_df = pd.DataFrame(data=ids[:len(df)], columns=['id'])
df = pd.concat([str_df, df], axis=1)

df


,id,0,1,2,3,4,5,6,7,8,...,990,991,992,993,994,995,996,997,998,999
0,"tf.Tensor(b'0R3102@37', shape=(), dtype=string)",8.592501e-08,0.000013,5.052114e-08,3.195686e-08,1.761134e-07,0.000001,0.000003,5.550361e-08,6.964341e-08,...,2.439221e-07,7.601395e-07,7.535504e-07,1.125314e-08,5.508833e-08,7.129789e-08,2.118278e-07,1.240013e-07,0.000019,0.001368
1,"tf.Tensor(b'004203@29', shape=(), dtype=string)",8.733767e-07,0.000098,4.317778e-06,2.617856e-06,1.659974e-05,0.000022,0.000034,8.624845e-06,5.508620e-06,...,5.587551e-06,3.296817e-05,4.890794e-06,1.488187e-06,2.060314e-06,6.585848e-06,1.172571e-05,3.555988e-06,0.000116,0.000642
2,"tf.Tensor(b'000153@45', shape=(), dtype=string)",1.808555e-06,0.000156,1.154085e-05,8.285621e-06,9.905442e-05,0.000051,0.000303,4.542567e-06,2.338888e-06,...,5.947137e-06,2.983239e-06,4.343022e-06,2.605895e-07,2.075952e-06,6.060913e-06,6.394870e-06,1.612421e-06,0.000053,0.002850
3,"tf.Tensor(b'00034B@34', shape=(), dtype=string)",3.055058e-06,0.000108,8.623540e-06,4.413317e-07,4.081579e-06,0.000005,0.000003,2.083445e-06,1.499429e-05,...,1.287251e-05,4.714233e-06,2.084579e-06,7.533853e-08,3.221979e-06,5.318400e-06,1.808323e-06,1.033239e-06,0.000174,0.000431
4,"tf.Tensor(b'0W7501@37', shape=(), dtype=string)",1.392959e-06,0.000107,8.795612e-06,6.301270e-06,4.045221e-05,0.000103,0.000216,1.455596e-06,2.757848e-06,...,3.946844e-07,7.021702e-06,5.132617e-06,3.821758e-07,2.052676e-06,2.836372e-06,5.392584e-06,3.022833e-07,0.000066,0.004987
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24095,"tf.Tensor(b'050031@51', shape=(), dtype=string)",2.489498e-05,0.000150,5.301214e-05,2.697824e-05,2.819716e-04,0.000236,0.000593,1.411668e-05,1.110802e-05,...,2.508990e-05,2.351928e-05,2.151933e-05,6.890675e-06,5.336854e-05,3.115540e-05,8.847587e-05,1.411043e-05,0.000196,0.023919
24096,"tf.Tensor(b'074910@23', shape=(), dtype=string)",8.552543e-06,0.000020,2.104495e-06,4.557573e-06,8.886963e-06,0.000278,0.000060,1.918233e-06,7.907082e-07,...,8.971779e-06,9.363106e-06,4.235483e-06,4.447344e-07,1.471125e-06,5.323484e-07,2.322324e-06,6.334195e-07,0.000038,0.000011
24097,"tf.Tensor(b'020180@51', shape=(), dtype=string)",1.100554e-05,0.000192,8.968704e-06,2.857802e-06,1.027427e-05,0.000023,0.000037,5.325901e-06,3.879268e-06,...,1.018668e-05,2.355310e-05,4.339830e-05,8.689537e-07,6.812062e-06,8.698006e-06,1.274531e-05,4.338823e-06,0.000255,0.002569
24098,"tf.Tensor(b'007185@25', shape=(), dtype=string)",6.137823e-07,0.000057,1.434067e-05,2.821281e-06,1.799784e-05,0.000013,0.000025,7.508373e-07,2.129458e-06,...,4.879219e-06,1.173382e-06,3.479812e-06,1.178627e-07,4.322090e-07,4.091111e-06,1.047419e-06,7.801332e-07,0.000032,0.001757


In [ ]:
df.to_csv('vgg6.csv')

In [ ]:
baseModel = load_cnn()
baseModel.trainable = False ## Not trainable weights
dense_layer_1 = Dense(64, activation='relu')
dense_layer_2 = Dense(32, activation='relu')
dense_layer_3 = Dense(16, activation='relu')
model = Sequential([
    baseModel,
    dense_layer_1,
    dense_layer_2,
    dense_layer_3,
    Dense(1)  # Output layer for regression
])